# Who is the Kevin Bacon of the NHL?

Many of us are familiar with the [Six Degrees of Kevin Bacon](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) game. Pick an actor, then try to find a path of castmates to connect them to the actor Kevin Bacon. The number of connections in the shortest path it takes to connect that actor to Mr. Bacon is said to be that actor's "Bacon number". Thus, Tom Hanks has a Bacon number of 1, because he was in Apollo 13 with KB. Alfred Hitchcock has a Bacon number of 2 (he appeared in his movie The Birds, which had actress Tippi Hedren, who appeared with KB in the movie Jayne Mansfield's Car). So does [noted Auston Matthews friend](https://www.instagram.com/p/B6kXtplnV3D/?utm_source=ig_embed) Justin Bieber (he appeared with Kiefer Sutherland in Zoolander 2, who himself appeared with KB in the original 1990 Flatliners). Charlie Chaplin is a 3 (in Monsieur Verdoux with Marjorie Bennett, who was in Charley Varrick with Joe Don Baker, who was in Criminal Law with KB; yeah, I don't know any of those people or movies either, thank you [Oracle of Bacon](https://oracleofbacon.org/)). KB is a prolific actor. He has appeared in close to 70 movies, and his career has seen him appear with a wide range of actors. Indeed, it's quite difficult to find an actor whose Bacon number is more than a 3, and this is where the fun of the game comes in. Trying to piece together careers and collborations to figure out the answer is an interesting diversion for the most obsessed of movie buffs. And the real fun part is when someone makes an obscure connection that makes everyone else go, "X WAS IN WHAT MOVIE?!?!?!"

I've recently come to learn about graph theory, a branch of mathematics that is used to answer questions related to, among other things, social networks. I immediately thought of the Six Degrees game and its more obscure relative: the [Erdos number](https://en.wikipedia.org/wiki/Erd%C5%91s_number). And then I started to wonder who would the equivalent person in other social networks be. Hockey in particular has a reputation for being a [small world](https://nationalpost.com/sports/hockey/nhl/calgary-flames/flames-pto-macdonalds-connection-to-straschnitzki-family-proves-hockey-is-a-small-world/wcm/a052253e-5c79-4fe0-a53c-87e08d80bde7).  While it is widely-accepted among hockey fans that the Toronto Maple Leafs are in fact the centre of the hockey universe, it would be interesting to see which actual player sits closest to that.

## Data, Setup, and Rules

I used the [NHL website's API](https://gitlab.com/dword4/nhlapi) to download data on all games that were ever played in the history of the NHL. I also used it to download data on all players who were part of the roster of those games. The data I used runs from the 1917-18 season through the 2017-18 season. 100 years seems like very appropriate bookends for this analysis.

To connect 2 players with each other, they must have simply played 1 regular season NHL game with each other. No playoffs, no preseason games, and definitely no All-Star games. The two players need to have been teammates. Given that, I toyed with only considering players who played a significant number of games with each other (a half-season or the like) to be connected, but I think this goes against the spirit of connection games. Like I mentioned before, extremely obscure trivia is after all where the fun of these games lies. Keeping the standard lax ensures that's maximized.

There are some things that would help a player be more "central":

* They have played a large number of games in the NHL. Immediately, that gives an advantage to players who have had longevity. Playing a 1000 games helps, playing 1200 helps a lot, and playing 1500 games almost guarantees being near the top of this list.

* Playing on many different teams is also really helpful. Think about it: if a player signs a UFA contract to stay with their current team, their teammates the following year will largely be the same. However, if they sign with a completely different team, chances are they won't have played with any of the players on that new team at the professional level. This is a much bigger opportunity for creating new connections. Players who get traded a lot or those that signed with several different teams as free agents will get the benefit of this.

* Playing with other central players is also extremely helpful. If a player X played with Y, and Y played with A,B,C,D, and E, those are all now 2nd degree connections of X. A player who plays with several players like this without those players playing with each other will be classified as more central. This is not exactly what we're looking for, and I will be looking for a way to detect that. There's no real science here, but it feels more interesting to find someone who has truly played with many players.

From the above, I can think of a couple of names that seem to fit there requirements:

* Jaromir Jagr, who played 1733 games in the NHL over 24 years with 9 different teams

* Mike Sillinger, who played in over a 1000 games with 12 different teams. He was traded a whopping 9 times (both of those are NHL records)

* ~~Toronto Maple Leafs~~ ~~St. Louis Blues~~ Florida Panthers legend Olli Jokinen: 1200+ games, 7 teams

So let's see if any of those guesses holds up! The rest of this notebook goes through my code. If you don't want to muck about with that, you can skip ahead to the results to see what I find.

## General Strategy

My "scrape_data.py" script was used to scrape the NHL website of player and game data. I will use this data to create a bipartite graph with player and game partitions. Each game will have two nodes, one for the home team and one for the away team, and player nodes will be linked to those game nodes as appropriate. I will then project this bipartite graph onto an undirected, unweighted graph with no partitions containing only player nodes and the edges that connect them. Using this graph, I will calculate the length of the shortest paths between every node pair. I will then use those lengths to find the average shortest path of each node and thus the most "central" player.

You can see an example of what the bipartite graph will look like. This allows us to match only players who have played games with each other. These players played in the game between the Pittsburgh Penguins and the Ottawa Senators on December 5th, 2016. Notice that there are two game nodes associated with this game, one for the home team Penguins and another for the visiting Senators. This is because I am linking only players who have played with one another on the same team. 

<img src='../reports/figures/bipartite.png' alt='Illustration of bipartite'>

I then project the bipartite graph onto the player nodes to create a simple undirected graph. The new graph's nodes are for players only. Any players that were connected to the same game node in the bipartite graph will now be connected to each other in the new graph. 

<img src='../reports/figures/bipartite2.png' alt='Projected graph'>

(Of course, in our actual graph, Phil Kessel and Dion Phaneuf would also be connected.)

In [1]:
import pandas as pd
import networkx as nx
import os
import urllib
import json
import pickle

I first piece together a dataframe of all NHL games played, which also includes the home and away teams in those games. I add two columns to those dataframes that contain the names I will assign to the nodes, one for the home team's players and the other for the away team's players.

In [2]:
data_path = '../data/raw/'
game_lists = os.listdir(os.path.join(data_path,'game_lists'))
player_lists = os.listdir(os.path.join(data_path,'player_lists'))

games = pd.concat(
    [pd.read_csv(os.path.join(data_path,'game_lists',f))
     for f in game_lists],
    ignore_index=True,
    sort=False
)

games['away_node_name'] = games['gamePk'].astype(str) + 'a'
games['home_node_name'] = games['gamePk'].astype(str) + 'h'

# games.to_csv('../data/interim/games.csv',index=False)

# games = pd.read_csv('../data/interim/games.csv')

I construct the players dataframe from the player files for each season. I then merge players on the games codes first for players who played away games then players who played home games. I then remove player-games where the player did not actually play or was healthy scratched. This removes backup goalies who did not play during the game, as well as scratched players.

In [3]:
# players = pd.concat(
#     [pd.read_csv(os.path.join(data_path,'player_lists',f))
#      for f in player_lists],
#     ignore_index=True,
#     sort=False
# )

# players = (players
#            .merge(games[['away_id','gamePk','away_node_name']],
#                   left_on=['team_id','game_id'],
#                   right_on=['away_id','gamePk'],
#                   how='left'))
# players = (players
#            .merge(games[['home_id','gamePk','home_node_name']],
#                   left_on=['team_id','game_id'],
#                   right_on=['home_id','gamePk'],
#                   how='left'))
# players['node_name'] = (players['away_node_name']
#                         .combine_first(players['home_node_name']))

# players = players.loc[~(players['code'].isin([float('NaN')]))]
# players = players[(players['game_id'].astype('str').apply(lambda x:x[5])=='2')]
# players.to_csv('../data/interim/players.csv',index=False)

players = pd.read_csv('../data/interim/players.csv')

Now that I have all player games, I can create the bipartite graph and then project it onto the the player nodes. By doing a sanity check on the number of unique players that have ever played in the league, I arrive at 7710 players, compared to 7731 players according to the NHL website. I arrived at this number by using the NHL website's [statistics page](http://www.nhl.com/stats/skaters), where searching for all players by season from 1917-18 to 2017-18 for the regular season yields 6945 skaters and 786 goalies. I think this is close enough for our purposes, though of course I would have preferred to have had an identical match of players.

In [4]:
graph = nx.Graph()


graph.add_nodes_from(players['id'],bipartite='players')
graph.add_nodes_from(players['node_name'],bipartite='games')

graph.add_edges_from(list(players[['id','node_name']]
                          .itertuples(index=False,name=None)))

player_nodes = list(x for x,y in graph.nodes(data=True) if y['bipartite']=='players')
print(len(player_nodes))

proj_graph = nx.algorithms.bipartite.projection.projected_graph(graph,player_nodes)

# I save the graph as a json file to allow accessing it again if need be.
json_graph = nx.readwrite.json_graph.node_link_data(proj_graph)
with open('../data/processed/proj_graph.json','w') as f:
    json.dump(json_graph,f)

7710


For each node, I will calculate two measures to help us find central players. First, I will determine the degree centrality of each player. The higher the degree, the more connected a player is to others in the network. This helps us find players that have appeared in many games with lots of different players. I will also calculate the length of the shortest path between every pair of players. This is by definition the degree distance between each player pair. Two players X and Y that played with each other have a shortest path length of 1, and thus player X's Y number is 1, and vice versa. If player W played with player X who played with Y who played with Z, the distance between W and Z is 3 and W's Z Number is 3 and vice versa. More "central" players will have shorter paths to other players/ Thus I would like to find the player with the shortest average distance to other nodes.

In [5]:
degree = {k:len(list(proj_graph.neighbors(k))) for k in proj_graph.nodes()}

In [6]:
# %time lengths = {node:dict(nx.algorithms.shortest_paths.unweighted.single_target_shortest_path_length(proj_graph,node)) for node in proj_graph.nodes()}

# I will save the result as a json file as calculating all the shortest distances takes a long time. 
# For example, running the algorithm on my machine took 50 minutes.

# with open('data/interim/lengths.json','w') as f:
#     json.dump(lengths,f)

with open('../data/interim/lengths.json','r') as f:
    lengths = json.load(f)

After calculating the shortest path distances between every pair of players, I can use the resulting data to calculate the average shortest distance between each one node and every other node in the graph. Networkx returns all distances including between the node and itself (which is 0), which is why we have to subtract 1 from the denominator of the average.

In [7]:
average_path_lengths = {k:sum([v for v in lengths[k].values()])/(len(lengths[k].values())-1) for k in lengths.keys()}
average_length_df = pd.DataFrame.from_dict(average_path_lengths,orient='index')
average_length_df.index = average_length_df.index.astype(int)

distance_df = average_length_df.merge(players[['id','fullName']],left_index=True,right_on='id').drop_duplicates('id')
distance_df.rename(columns={0:'distance'},inplace=True)
distance_df = distance_df.merge(pd.DataFrame.from_dict(degree,orient='index').rename(columns={0:'degree'})
                                ,left_on='id',
                                right_index=True)

distance_df['distance_rank'] = distance_df.sort_values('distance')['distance'].rank()

distance_df.to_csv('../data/processed/avg_distance.csv',index=False)

# distance_df = pd.read_csv('../data/processed/avg_distance.csv')

## The Centre of the NHL Player Universe

In [8]:
distance_df.sort_values('distance').iloc[:20]

,distance,id,fullName,degree,distance_rank
509170,2.429628,8449573,Mark Messier,416,1.0
509384,2.456090,8450905,Gordie Roberts,328,2.0
599862,2.478921,8447989,Phil Housley,433,3.0
599878,2.485018,8445000,Dave Andreychuk,391,4.0
575706,2.486185,8446951,Ron Francis,360,5.0
855832,2.489298,8448208,Jaromir Jagr,513,6.0
539448,2.491633,8446117,Paul Coffey,391,7.0
509379,2.493319,8445621,Ray Bourque,313,8.0
509390,2.495265,8448002,Mark Howe,239,9.0
798701,2.501232,8450725,Mark Recchi,476,10.0


And there's an interesting result! It turns out, Mark Messier is the most "accessible" player in the history of the NHL. Put simply, it takes on average 2.43 steps to connect any player in the history of the league to Messier. He is followed by Gordie Roberts, a US Hockey Hall of Famer who player close to 1100 games in the NHL spending a large portion of his career with the Minnesota North Stars. My initial guesses of Jagr and Sillinger are not too far off, with some interesting players who have been around the league in Doug Gilmour and Marc Bergevin. Where did Jokinen place?

In [9]:
distance_df['distance_rank'] = distance_df.sort_values('distance')['distance'].rank()
distance_df[distance_df['fullName'] == 'Olli Jokinen']

,distance,id,fullName,degree,distance_rank
1106481,2.715916,8466140,Olli Jokinen,445,603.5


That's not very close to the top.

The other measure to look at is the degree of each player, that is, the number of links a node has with other nodes. It can be thought of as the number of unique teammates that this player has had in their career. 

In [10]:
distance_df.sort_values('degree',ascending=False).iloc[:20]

,distance,id,fullName,degree,distance_rank
855832,2.489298,8448208,Jaromir Jagr,513,6.0
855669,2.501751,8451359,Mike Sillinger,493,11.0
759454,2.538462,8451224,Mathieu Schneider,477,38.0
798701,2.501232,8450725,Mark Recchi,476,10.0
1112201,2.771566,8464989,Matt Cullen,446,958.5
1106481,2.715916,8466140,Olli Jokinen,445,603.5
665637,2.518744,8445428,Marc Bergevin,441,19.0
759566,2.550006,8450824,Luke Richardson,435,49.0
599862,2.478921,8447989,Phil Housley,433,3.0
920828,2.598132,8458537,Ray Whitney,429,146.0


That result is more in line with my expectations. Jagr, having played 24 years on 9 teams, and Sillinger, having played on 12 teams, top our list. Long-time NHL'er Matt Cullen is also up there, having played more than 1500 games by the time he retired at the end of the 2018-19 season. Jokinen follows close behind. And Messier, near the top of the all-time GP in the NHL, remains near the top of the pack by degrees.

## So who *is* the Kevin Bacon of the NHL?

There's no real answer to that question. As I said before, I think the real fun of six degrees games is the trivia it brings to light and the fun of piecing together the careers of the people in question. The Six Degrees of Kevin Bacon game reportedly [started out similarly, with several friends snowed in, drunk, and watching TV](https://www.npr.org/2012/09/14/161130024/the-history-of-six-degrees-of-kevin-bacon) who had then realized just how many movies KB had been in. The choice of KB, while certainly very good, was coincidental. So it feels wrong to try to call any one player a centre of the NHL player universe, and I think it goes to the reader to decide for themselves who that person should be.

To that end, I wrote a snippet of code below to find a shortest path between any two players in the history of the league. Simply enter the names of the players and the code will produce the trail of players that connect the two.

In [11]:
# distance_df = pd.read_csv('../data/processed/avg_distance.csv')
# with open('../data/processed/proj_graph.json','r') as f:
#     proj_graph = nx.readwrite.json_graph.node_link_graph(json.load(f))

source_name = 'Bill Barilko'
target_name = 'Auston Matthews'

source = distance_df.loc[distance_df['fullName']==source_name,'id'].values[0]
target = distance_df.loc[distance_df['fullName']==target_name,'id'].values[0]
path = nx.algorithms.shortest_paths.generic.shortest_path(proj_graph,source,target)

pd.DataFrame({'id':path}).merge(distance_df,on='id')[['id','fullName']]

,id,fullName
0,8445019,Bill Barilko
1,8444971,George Armstrong
2,8447664,Jim Harrison
3,8449573,Mark Messier
4,8468575,Dominic Moore
5,8479318,Auston Matthews


## The Loneliest Players in the NHL's History

Let's look at the opposite end of the spectrum. Who are the "least" connected players in the history of the league?

In [12]:
distance_df.sort_values('distance',ascending=False).iloc[:30]

,distance,id,fullName,degree,distance_rank
1188,6.327410,8447816,George McNaughton,7,7710.0
1252,6.327280,8449382,Alex Wellington,8,7709.0
1224,6.326891,8449087,Tommy Smith,10,7708.0
1124,6.326501,8449843,Frank Brophy,13,7707.0
1679,6.323907,8447796,Fred McLean,9,7706.0
173,6.279154,8448119,Evariste Payer,9,7705.0
622,6.276690,8445914,Fred Doherty,10,7704.0
1110,6.275392,8447815,Howard McNamara,10,7703.0
1115,6.275003,8449072,Don Smith,11,7702.0
2660,6.249060,8445317,Dave Campbell,10,7701.0


We here find mostly a list of almost-forgotten names who had been part of the NHL briefly at one point or another during its formative years. Bernie Morris had a 20 year career around several leagues like the PCHA and the minor Canadian Professional Hockey League, including 6 games with the Boston Bruins. Art Ross, for whom the trophy is named, had a similarly long career, finishing it with 3 games with the Montreal Wanderers in the league's first season. Fred Doherty played mostly before the NHL was founded, including one game with the Toronto Blueshirts, whose ownership's hostilities with other team owners of the NHA led to the foundation of the NHL. Bob Hall played 8 games with the New York Americans for a month in 1926. And all the way at the top, the least central, the loneliest player in the history of the NHL, George McNaughton, who played one game for the Quebec Bulldogs, on January 1st, 1918, against the original Ottawa Senators.

He is recorded to have scored no points.

But if we look closely at this list, we see something interesting. Even this list of mostly forgotten players has an interesting aspect: the farthest average distance any of those players have from any other player in the history of the league is not much larger than 6.

The [theory of the six degrees of separation](https://en.wikipedia.org/wiki/Six_degrees_of_separation) lives on!

## How isolated are the most isolated players?

Speaking of isolation, what is the maximum number of association jumps it would take to move from one player to another? To determine this, for each node we can calculate a measure called eccentricity that tells us what is the length of the longest shortest path between the node and all other nodes in the graph. Then we can find the maximum value of eccentricities from all players to get our answer. 

In [13]:
eccentric = {k:max(lengths[k].values()) for k in lengths.keys()}

eccentric_df = pd.DataFrame.from_dict(eccentric,orient='index').rename(columns={0:'eccentricity'})
eccentric_df['eccentricity'].max()

8

This is, to me, a very interesting result! To go from any one player to another, we need not make more than 8 jumps. This speaks to the very connected nature of the league and how small social networks can truly be. Even to go from our least connected players to any other player, it doesn't take all that much movement!

## Who are the central players if we only look at modern players?

Asking others to comment on who they think would be at the centre of the NHL, I got the sense that most people are more focused on the present and players who are currently playing or only recently retired. So I re-did my analysis only looking at players who had their rookie season in the 1990-91 season. This is because that's the rookie year of the oldest player in the league in 2017-18 (our last year of data), that player being Jagr. Once that's done, we see some more familiar names. 

In [14]:
players['season'] = players['node_name'].str[:4].astype(int)
recent_players = players.groupby('id')['season'].min()
recent_players = recent_players[recent_players>=1990]
players = players[players['id'].isin(recent_players.index)]

degree = {k:len(list(proj_graph.neighbors(k))) for k in proj_graph.nodes()}

In [15]:
# %time lengths = {node:dict(nx.algorithms.shortest_paths.unweighted.single_target_shortest_path_length(proj_graph,node)) for node in proj_graph.nodes()}

# Again, I save the result as a json file as these calculations do still take a significant amount of time,
# though not as much as the original set of distance calculations

# with open('data/interim/lengths_recent.json','w') as f:
#     json.dump(lengths,f)

with open('../data/interim/lengths_recent.json','r') as f:
    lengths = json.load(f)

In [16]:
average_path_lengths = {k:sum([v for v in lengths[k].values()])/(len(lengths[k].values())-1) for k in lengths.keys() if len(lengths[k].values())!=1}
average_length_df = pd.DataFrame.from_dict(average_path_lengths,orient='index')
average_length_df.index = average_length_df.index.astype(int)

distance_df = average_length_df.merge(players[['id','fullName']],left_index=True,right_on='id').drop_duplicates('id')
distance_df.rename(columns={0:'distance'},inplace=True)
distance_df = distance_df.merge(pd.DataFrame.from_dict(degree,orient='index').rename(columns={0:'degree'})
                                ,left_on='id',
                                right_index=True)
distance_df.to_csv('../data/interim/avg_distance_recent.csv',index=False)

In [17]:
distance_df.sort_values('distance').iloc[:30]

,distance,id,fullName,degree
1106481,1.927119,8466140,Olli Jokinen,445
855832,1.928175,8448208,Jaromir Jagr,513
1112201,1.933721,8464989,Matt Cullen,446
1379609,1.940322,8468575,Dominic Moore,404
1025153,1.945603,8462041,Radek Dvorak,418
1002419,1.955374,8458637,Sean O'Donnell,408
1189658,1.960391,8467831,Jeff Halpern,368
922547,1.967256,8458529,Alex Kovalev,394
1005530,1.968577,8458951,Sergei Gonchar,376
1147363,1.969105,8467334,Manny Malhotra,365


## One Last Interesting Piece of Trivia

In our players list, I tried to look for some interesting players who had played on teams we would not expect. To that end, I came across this:

In [18]:
players = pd.read_csv('../data/interim/players.csv')
players[players['team_name']=='Detroit Red Wings'].groupby('fullName').filter(lambda x: len(x)>10).groupby('fullName')['id'].count().sort_values().iloc[:20]

fullName
Erik Cole           11
Hec Lalande         11
Dave Hanson         11
Rick Smith          11
Dave Trottier       11
Stu McNeill         11
Kent Huskins        11
Dmitri Mironov      11
Mal Davis           11
Tord Lundstrom      11
Wendel Clark        12
Dave Silk           12
Bob McGill          12
Gene Achtymichuk    12
Jack Keating        12
Joe Lamb            12
Bill Folk           12
Edward Diachuk      12
Murray Hall         13
Larry Gloekner      13
Name: id, dtype: int64

Wendel Clark played for the Red Wings...??? It turns out, in a bit of forgotten history, yes. Leafs legend Wendel Clark played for the Red Wings in the tail-end of the 1998-99 season.

But it gets better.

Clark had initially started the season with the Tampa Bay Lightning, having signed with them as a free-agent. He played well, scoring 28 goals and earning a spot at the All-Star game. But Tampa was very bad that year, eventually finishing dead last, and traded Clark along with other pieces at the trade deadline. In return, they received goalie Kevin Hodson and a 2nd round pick in the 1999 draft.

The Lightning used that pick to draft current Maple Leafs coach Sheldon Keefe.